In [1]:
import pandas as pd
import json
import cv2
import numpy as np
from tqdm import tqdm

In [2]:
# Load dataset from train sample videos json file
def load_json_into_df(file:str):
    files = []
    file_path = f"../../train_sample_videos/{file}"
    with open(file_path) as f:
        data = json.load(f)
        for x in data:
            files.append({
                "name":x,
                "label":data[x]["label"],
                "split":data[x]["split"],
                "original":data[x]["original"]
            })
    return pd.DataFrame(files)

In [3]:
df = load_json_into_df("metadata.json")

In [4]:
df.head()

,name,label,split,original
0,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
1,aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
2,abarnvbtwb.mp4,REAL,train,None
3,abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
4,abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4


In [5]:
def extract_frames(video_path:str, num_frames:int=10):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    # Get the total number of frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Determine no of frames to extract
    # num_frames = total_frames if num_frames > total_frames else num_frames


    # Compute the frame indices to extract
    frame_indices = np.linspace(0, total_frames - 1, num=num_frames, dtype=int)


    # Initialize an empty list to store the frames
    frames = []


    # Loop through the selected frame indices and extract the frames
    for index in frame_indices:
        # Set the frame index
        cap.set(cv2.CAP_PROP_POS_FRAMES, index)

        # Read the frame from the video file
        ret, frame = cap.read()

        # Convert the frame to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Add the frame to the list
        frames.append(frame)


    # Release the video file
    cap.release()


    # Return the list of frames as a NumPy array
    return np.array(frames)

In [6]:
def extract_faces(frame):
    face_cascade = cv2.CascadeClassifier('../../models/haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(
        frame,
        scaleFactor= 1.1,
        minNeighbors= 20,
        minSize=(10, 10)
    )
    faces_detected = format(len(faces)) + " faces detected!"
    if len(faces_detected) == 0:
        return
    # Draw a rectangle around the faces
    faces_frames = []
    for (x,y,w,h) in faces:
        face = frame[y:y+h, x:x+w]
        faces_frames.append(face)
    return faces_frames

In [7]:
def extract_frames_faces(df:pd.DataFrame):
    train_df_rows = []
    for index, row in tqdm(df.iterrows()):
        video_path = f"../../train_sample_videos/{row['name']}"
        frames =extract_frames(video_path)
        for frame in frames:
            faces = extract_faces(frame)
            for face in faces:
                train_df_rows.append({
                    "video_name":row["name"],
                    "label":row["label"],
                    "split":row["split"],
                    "original":row["original"],
                    "raw_face":face
                })

    return pd.DataFrame(train_df_rows)

In [8]:
train_df = extract_frames_faces(df)

400it [20:24,  3.06s/it]


In [9]:
train_df.head()

,video_name,label,split,original,raw_face
0,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4,"[[[117, 109, 91], [123, 115, 97], [112, 104, 8..."
1,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4,"[[[140, 165, 120], [136, 155, 112], [130, 150,..."
2,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4,"[[[136, 172, 96], [117, 153, 77], [120, 153, 8..."
3,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4,"[[[116, 154, 88], [116, 154, 90], [117, 155, 9..."
4,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4,"[[[198, 224, 252], [198, 224, 250], [196, 222,..."


In [12]:
train_df.to_pickle('../../preprocessing/train.pkl')

In [11]:
# cv2.imshow("image", train_df.iloc[0]["raw_face"])
# cv2.waitKey(0)
#
# # closing all open windows
# cv2.destroyAllWindows()

In [13]:
train_df.head()

,video_name,label,split,original,raw_face
0,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4,"[[[117, 109, 91], [123, 115, 97], [112, 104, 8..."
1,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4,"[[[140, 165, 120], [136, 155, 112], [130, 150,..."
2,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4,"[[[136, 172, 96], [117, 153, 77], [120, 153, 8..."
3,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4,"[[[116, 154, 88], [116, 154, 90], [117, 155, 9..."
4,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4,"[[[198, 224, 252], [198, 224, 250], [196, 222,..."
